In [37]:
import geopandas as gpd
import pandas as pd

## Open State Data

In [38]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)
states_df = states_df.to_crs(9311)

## Open Population Data

In [39]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [40]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [41]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Open Aviation Facilities Data

In [42]:
file_path = "data/NTAD_Aviation_Facilities_-4386985993802120360.gpkg"
airports_gdf = gpd.read_file(file_path)

In [43]:
airports_gdf = airports_gdf[
    airports_gdf["FAR_139_CARRIER_SER_CODE"] == "S"
].reset_index()
airports_gdf = airports_gdf.to_crs(9311)
airports_gdf.to_file("data/airports.gpkg")

In [44]:
airports_counts_df = (
    pd.DataFrame(airports_gdf.groupby("COUNTY_ASSOC_STATE").size())
    .reset_index()
    .rename(columns={"COUNTY_ASSOC_STATE": "STUSPS", 0: "AIRPORTS"})
)

## Merge data

In [45]:
airports_counts_gdf = states_with_population_df[
    ["NAME", "STUSPS", "geometry", "POPULATION"]
].merge(airports_counts_df, on="STUSPS", how="inner")

In [46]:
airports_counts_gdf["sqmi"] = airports_counts_gdf["geometry"].area / 2589988.11
airports_counts_gdf["AIRPORTS_PER_10_SQMI"] = airports_counts_gdf["AIRPORTS"] / (
    airports_counts_gdf["sqmi"] / 10.0
)

In [47]:
airports_counts_gdf["per_100k"] = (
    (airports_counts_gdf["AIRPORTS"] / (airports_counts_gdf["POPULATION"] / 100_000))
    .round(decimals=2)
    .astype(float)
)

## Export

In [48]:
airports_counts_gdf = airports_counts_gdf.dropna()
airports_counts_gdf.to_file("data/airports_per_state.gpkg")